
## 1. Requerimientos Generales (aplican a todos los módulos)

- **RG-1 (Lenguaje):** Todos los programas deben implementarse en **Python ≥ 3.8**.  
- **RG-2 (Librerías):** Se deberá utilizar **NumPy** para operaciones numéricas y manejo de arreglos; **Pandas** para carga y escritura de datos `.csv`
- **RG-3 (Claridad):** Cada archivo `.py` deberá incluir:  
  - Encabezado con descripción del propósito.  
  - Comentarios claros para cada función.  
  - Nombres de variables autoexplicativos.  
- **RG-4 (Reproducibilidad):** El código debe permitir replicar los resultados sin modificaciones adicionales al entorno o parámetros.  
- **RG-5 (Gestión de errores):** Validar existencia y consistencia de los archivos `.csv` antes de procesarlos.  

---

## 2. Requerimientos de `ppr.py` (Preprocesamiento de Datos)

- **RPPR-1:** Implementar función `get_features()` para calcular entropía:  
  - Modo Dispersión → parámetros: `d`, `tau`, `c`.  
  - Modo Permutación → parámetros: `d`, `tau`.  
- **RPPR-2:** Leer los archivos de entrada: `class1.csv` y `class2.csv`.  
- **RPPR-3:** Segmentar los datos en ventanas de tamaño `W` (filas).  
- **RPPR-4:** Generar archivos de salida con características:  
  - `dfeatures1.csv` → características extraídas de `class1.csv`.  
  - `dfeatures2.csv` → características extraídas de `class2.csv`.  
- **RPPR-5:** Concatenar ambos conjuntos de características en un solo archivo:  
  - `dfeatures.csv` → de dimensión `(2K, W)`.  
- **RPPR-6:** Crear archivo de etiquetas `label.csv`:  
  - Clase 1 (`K/2` filas) → etiqueta `1`.  
  - Clase 2 (`K/2` filas) → etiqueta `0`.  
- **RPPR-7:** Incluir función `main()` que ejecute de forma secuencial:  
  1. Lectura de datos.  
  2. Generación de características.  
  3. Concatenación y guardado.  
  4. Creación de etiquetas.  

---

## 3. Requerimientos de `trn.py` (Entrenamiento y Evaluación)

- **RTRN-1:** Cargar datos desde `dfeatures.csv` y `label.csv`.  
- **RTRN-2:** Implementar un modelo de **Regresión Logística** entrenado mediante **mini-batch Gradient Descent (mGD)**.  
- **RTRN-3:** Permitir ajuste de hiperparámetros:  
  - Tasa de aprendizaje (`α`).  
  - Tamaño de batch (`m`).  
  - Número máximo de iteraciones.  
- **RTRN-4:** Entrenar el modelo utilizando el conjunto de entrenamiento y guardar parámetros finales (pesos y bias).  
- **RTRN-5:** Evaluar el modelo con conjunto de prueba:  
  - Calcular matriz de confusión.  
  - Calcular F1-score para cada clase binaria.  
- **RTRN-6:** Generar reporte de rendimiento en pantalla o archivo (`results.txt`).  
- **RTRN-7:** Incluir función `main()` que ejecute de forma secuencial:  
  1. Carga de datos.  
  2. Entrenamiento del modelo.  
  3. Evaluación del modelo.  
  4. Presentación de resultados.  

---

## 4. Requerimientos de `utils.py` (Opcional, Librería de Apoyo)

> Este módulo es opcional, pero recomendable para mantener el código modular y reutilizable.  

- **RUTIL-1:** Implementar funciones auxiliares como:  
  - `save_data(data, filename)` → guardar `.csv`.  
  - `load_data(filename)` → cargar `.csv`.  
  - `plot_confusion_matrix(cm)` → graficar matriz de confusión.  
- **RUTIL-2:** Centralizar en este archivo todas las funciones comunes a `ppr.py` y `trn.py`.  
- **RUTIL-3:** Garantizar que el código principal (`ppr.py` y `trn.py`) se mantenga limpio y enfocado en lógica principal.  

---

## ppr.py


Propósito  
El propósito de este programa es crear nuevas características desde la data original utilizando la entropía de Dispersión y la entropía de Permutación.  

Función Principal  
Nombre de la función: `get_features()`  
Descripción: Esta función crea características de entropía a partir de los archivos `class1.csv` y `class2.csv`.  

Opciones de Cálculo de Entropía  
1. Opción Dispersión  
   - Parámetros:  
     - d: dimensión embedding  
     - tau: tiempo de retardo  
     - c: número de clases  

2. Opción Permutación  
   - Parámetros:  
     - d: dimensión embedding  
     - tau: tiempo de retardo  

Segmentación de Archivos  
- Los archivos de clase original deben segmentarse en ventanas de tamaño `W` (número de filas).  

Archivos de Salida  
1. `dfeatures1.csv`  
   - Derivado de `class1.csv`.  
   - W columnas: cada columna corresponde a una característica de entropía.  
   - K filas: cada fila representa una nueva muestra de características.  

2. `dfeatures2.csv`  
   - Derivado de `class2.csv`.  
   - W columnas: cada columna corresponde a una característica de entropía.  
   - K filas: cada fila representa una nueva muestra de características.  

3. `dfeatures.csv`  
   - Creado a partir de la concatenación de `dfeatures1.csv` y `dfeatures2.csv`.  
   - Tamaño resultante:  
     - 2K filas.  
     - W columnas.  

4. `label.csv`  
   - Archivo de etiquetas asociado a las clases.  
   - Clase 1: Y(1 : K/2) = 1  
   - Clase 2: Y(K/2 + 1 : N) = 0  

Estructura del Programa Principal (`main`)  
```python
def main():
    conf_entropy()
    load_data()
    F1 = get_features("class1.csv")
    F2 = get_features("class2.csv")
    F = np.concatenate((F1, F2), axis=0)
    save_data(F, "dfeatures.csv")
    save_labels("label.csv")
```

In [6]:
#----------------------------------------------
# Create Features by use 
# Dispersion Entropy and Permutation entropy
#----------------------------------------------

import pandas as pd
import numpy as np
# from utility import entropy_dispersion, entropy_permuta

#----------------------------------------------
# Load parameters Entropy
#----------------------------------------------
def conf_entropy():
    conf = pd.read_csv("config/conf_ppr.csv")
    opt  = conf.loc[0, "opt"]    # "dispersion" o "permutation"
    d    = int(conf.loc[0, "d"])
    tau  = int(conf.loc[0, "tau"])
    c    = int(conf.loc[0, "c"])
    W    = int(conf.loc[0, "W"])  # tamaño de ventana
    
    return (opt, d, tau, c, W)

#----------------------------------------------
# Load Data
#----------------------------------------------
def load_data(nFile):
    data = pd.read_csv(nFile, header=None).values
    return data

#----------------------------------------------
# Obtain entropy : Dispersión or Permutación
#----------------------------------------------
def gets_entropy(x, opt, d, tau, c):
    if opt == "dispersion":
        entr = entropy_dispersion(x, d, tau, c)
    elif opt == "permutation":
        entr = entropy_permuta(x, d, tau)
    else:
        raise ValueError("Unknown entropy option")
    return entr

#----------------------------------------------
# Obtain Features by use Entropy    
#----------------------------------------------
def gets_features(file, opt, d, tau, c, W):
    X = load_data(file)
    N, L = X.shape
    K = N // W  # número de segmentos
    
    F = []
    for k in range(K):
        block = X[k*W:(k+1)*W, :]
        feats = [gets_entropy(block[:, j], opt, d, tau, c) for j in range(L)]
        F.append(feats)
    F = np.array(F)
    return F    

#----------------------------------------------
# Save Features
#----------------------------------------------
def save_data(F, fname="dfeatures.csv"):
    pd.DataFrame(F).to_csv(fname, header=False, index=False)

def save_labels(K, fname="label.csv"):
    Y = np.concatenate((np.ones(K//2), np.zeros(K//2)))
    pd.DataFrame(Y).to_csv(fname, header=False, index=False)


In [7]:
"""
def main():
    opt, d, tau, c, W = conf_entropy()
    F1 = gets_features("data/class1.csv", opt, d, tau, c, W)
    F2 = gets_features("data/class2.csv", opt, d, tau, c, W)
    F  = np.concatenate((F1, F2), axis=0)
    save_data(F, "dfeatures.csv")
    save_labels(F.shape[0], "label.csv")
"""


opt, d, tau, c, W = conf_entropy()
F1 = gets_features("data/class1.csv", opt, d, tau, c, W)
F2 = gets_features("data/class2.csv", opt, d, tau, c, W)
F  = np.concatenate((F1, F2), axis=0)
save_data(F, "dfeatures.csv")
save_labels(F.shape[0], "label.csv")

KeyError: 'opt'

## trn.py


Etapa 2 – Algoritmo de Entrenamiento (trn.py)  

Propósito  
Este programa implementa el algoritmo de entrenamiento de un modelo de regresión logística, utilizando descenso de gradiente con momentum. Se encarga de dividir los datos en conjuntos de entrenamiento y prueba, entrenar el modelo y almacenar los resultados obtenidos.  

Carga y Preparación de Datos  
- Cargar archivos `dfeatures.csv` y `label.csv`.  
- Reordenar aleatoriamente la matriz de características y el vector de clases para evitar sesgos.  
- Crear función para dividir los datos en conjuntos de entrenamiento y prueba:  
  - Datos de Entrenamiento:  
    - Archivos de salida: `dtrn.csv` y `dtrn_label.csv`.  
    - Número de muestras: L = round(N * p / 100).  
    - p: porcentaje de entrenamiento (ej. 60 o 80).  
  - Datos de Prueba:  
    - Archivos de salida: `dtst.csv` y `dtst_label.csv`.  
    - Número de muestras: K = N – L.  

Función de Entrenamiento – trn_logistic()  
- Cargar parámetros de configuración desde archivo `conf_train.csv`.  
- Implementar algoritmo de **descenso de gradiente con momentum**.  
- Aprender y actualizar los pesos del modelo en cada iteración.  
- Guardar los resultados del entrenamiento en archivos:  
  - `pesos.csv` → matriz de pesos finales del modelo.  
  - `costo.csv` → vector de costos por iteración.  
    - Número de filas: máximo de iteraciones.  
    - Número de columnas: 1.  

Estructura del Programa Principal (main)  
```python
def main():
    conf_train()
    load_data()
    train()
    save_w_cost(W, Cost, 'pesos.csv', 'costo.csv')
```

In [ ]:
# Logistic Regression's Training :

import numpy      as np
import utility    as ut

#Save weights and Cost
def save_w_cost():
    ...
    return
#
def iniWs():
    ...
    return(W,V)
# Load config train for Regression
#
#Training by use mGD
def train():    
    
    ....
    return()
# Load data to train 
def load_data():
    
    
    return()
#
def conf_train(nFile):
    ....
    return()

# Beginning ...
def main():    
    conf_train()
    load_data()   
    train()             
    save_w_cost(W,Cost, 'pesos.csv','costo.csv')
       
if __name__ == '__main__':   
	 main()



SyntaxError: invalid syntax (3221381587.py, line 19)

## tst.py

Etapa 3 – Evaluación del Modelo (eval.py)  

Cargar data de test:  
- Cargar los coeficientes desde archivo `pesos.csv`.  
- Generar los valores estimados usando Regresión Logit.  

Crear archivos de resultados:  
- Matriz de Confusión → `cmatriz.csv`, matriz de dimensión (2,2).  
- F-scores → `fscores.csv`, vector de dimensión (1,2).  

Programa Principal:  
```python
def main():
    load_data()
    load_w()
    zv = forward(xv, W)
    cm, Fsc = metricas(yv, zv)
    save_measure(cm, Fsc, 'cmatrix.csv', 'fscores.csv')
```

In [ ]:
# Testing for Logistic Regresion
import numpy as np

def forward(xv,w):
    ...
    return(zv)
#
def measure(yv,zv):
    ...
    return(cmatrix,Fscores)
#
def save_measure(cm,Fsc,nFile1,nFile2):
    ...
    return()
# Load weight
def load_w(nFile):
    ...
    return(W)
# 
def load_data(nFile):
    ....
    return(x,y)
# Beginning ...
def main():			
	load_data()
	load_w()
	zv     = forward(xv,W)      		
	cm,Fsc = metricas(yv,zv) 	
	save_measure(cm,Fsc,'cmatrix.csv','Fscores.csv')		

if __name__ == '__main__':   
	 main()



## utility.py

## Entropía de Dispersión (DE) (PTT 6)

La entropía de dispersión es un método para cuantificar la incertidumbre de una serie temporal, utilizando patrones embebidos y simbolización discreta.

**Normalización**  
El vector de datos $X$ se normaliza entre 0 y 1:

$$
x_i^{norm} = \frac{x_i - \min(X)}{\max(X) - \min(X)}
$$

**Embedding**  
Se construyen vectores embebidos de dimensión $d$ y retardo $\tau$:

$$
X_i = \{x_i, x_{i+\tau}, \ldots, x_{i+(d-1)\tau}\}
$$

Número total de vectores:

$$
M = N - (d-1)\tau
$$

**Simbolización**  
Cada vector $X_i$ se transforma en símbolos discretos:

$$
Y_i = \mathrm{round}(0.5 + X_i \cdot c)
$$

**Probabilidades**  
La probabilidad de ocurrencia de cada patrón es:

$$
p_k = \frac{f_k}{M}, \quad k = 1,2,\ldots,c^d
$$

**Entropía de Dispersión**  
La entropía se define como:

$$
DE = -\sum_{k=1}^{c^d} p_k \log(p_k)
$$

**Entropía Normalizada**  

$$
DE_{norm} = \frac{DE}{\log(c^d)}
$$


#### Entropía de Permutación (PE) (PTT 4)

La entropía de permutación es un método para cuantificar la complejidad de una serie temporal a partir del orden relativo de sus valores, utilizando patrones ordinales.

**Embedding**  
Se construyen vectores embebidos de dimensión $d$ y retardo $\tau$:

$$
X_i = (x_i, x_{i+\tau}, \ldots, x_{i+(d-1)\tau})
$$

Número total de vectores:

$$
M = N - (d-1)\tau
$$

**Número de patrones posibles**  

$$
d!
$$

**Probabilidades**  
La probabilidad de ocurrencia de cada patrón es:

$$
p_k = \frac{f_k}{M}, \quad k = 1,2,\ldots,d!
$$

**Entropía de Permutación**  
La entropía se define como:

$$
PE = -\sum_{r=1}^{d!} p_r \log(p_r)
$$

**Entropía Normalizada**  

$$
nPE = \frac{PE}{\log(d!)}
$$


In [1]:

# My Utility : auxiliars functions

import pandas as pd
import numpy  as np

# ---------------------------------------------------------
# CResidual-Dispersion Entropy
# ---------------------------------------------------------
def entropy_dispersion(x, d, tau, c):
    """
    Calcula la Entropía de Dispersión (DE).
    Parámetros:
    x   : array-like, serie temporal
    d   : dimensión de embedding
    tau : retardo
    c   : número de símbolos
    """
    x = np.array(x, dtype=float)
    N = len(x)

    # Paso 1: Normalizar entre [0,1]
    x_norm = (x - np.min(x)) / (np.max(x) - np.min(x) + 1e-12)

    # Paso 2: Embedding
    M = N - (d - 1) * tau
    if M <= 0:
        raise ValueError("Serie demasiado corta para embedding")
    X_emb = np.array([x_norm[i:i + d * tau:tau] for i in range(M)])

    # Paso 3: Simbolización
    Y = np.round(0.5 + X_emb * c).astype(int)
    Y[Y < 1] = 1
    Y[Y > c] = c

    # Paso 4: Patrones (convertir cada vector en número base-c)
    patrones = []
    for y in Y:
        code = 0
        for j in range(d):
            code += (y[j] - 1) * (c ** j)
        patrones.append(code)
    patrones = np.array(patrones)

    # Paso 5: Frecuencias
    unique, counts = np.unique(patrones, return_counts=True)
    probs = counts / M

    # Paso 6: Entropía de Shannon
    entr = -np.sum(probs * np.log(probs + 1e-12))

    # Paso 7: Normalización
    entr = entr / np.log(c ** d)

    return entr


# ---------------------------------------------------------
# Permutation Entropy
# ---------------------------------------------------------
def entropy_permuta(x, m, tau):
    """
    Calcula la Entropía de Permutación (PE).
    Parámetros:
    x   : array-like, serie temporal
    m   : dimensión de embedding
    tau : retardo
    """
    x = np.array(x, dtype=float)
    N = len(x)

    # Paso 1: Embedding
    M = N - (m - 1) * tau
    if M <= 0:
        raise ValueError("Serie demasiado corta para embedding")
    X_emb = np.array([x[i:i + m * tau:tau] for i in range(M)])

    # Paso 2: Patrones ordinales
    patrones = []
    for row in X_emb:
        perm = tuple(np.argsort(row))  # orden relativo
        patrones.append(perm)
    patrones = np.array(patrones)

    # Paso 3: Frecuencias
    unique, counts = np.unique(patrones, axis=0, return_counts=True)
    probs = counts / M

    # Paso 4: Entropía de Shannon
    entr = -np.sum(probs * np.log(probs + 1e-12))

    # Paso 5: Normalización
    from math import factorial, log
    entr = entr / log(factorial(m))

    return entr


In [3]:
def test_entropies():
    np.random.seed(0)

    print("\n=== 1. Serie constante ===")
    x_const = np.ones(100)
    print("DE (const):", entropy_dispersion(x_const, d=3, tau=1, c=3))
    print("PE (const):", entropy_permuta(x_const, m=3, tau=1))

    print("\n=== 2. Serie aleatoria uniforme ===")
    x_rand = np.random.rand(500)
    print("DE (random):", entropy_dispersion(x_rand, d=3, tau=1, c=6))
    print("PE (random):", entropy_permuta(x_rand, m=3, tau=1))

    print("\n=== 3. Serie senoidal ===")
    t = np.linspace(0, 10*np.pi, 500)
    x_sin = np.sin(t)
    print("DE (sinusoidal):", entropy_dispersion(x_sin, d=3, tau=2, c=6))
    print("PE (sinusoidal):", entropy_permuta(x_sin, m=3, tau=2))

    print("\n=== 4. Sensibilidad a parámetros ===")
    x_test = np.random.randn(300)
    for d in [2, 3, 4]:
        print(f"DE (d={d}): {entropy_dispersion(x_test, d=d, tau=1, c=5):.4f}")
    for m in [2, 3, 4]:
        print(f"PE (m={m}): {entropy_permuta(x_test, m=m, tau=1):.4f}")

    print("\n=== 5. Comparación en señal mixta (sin+ruido) ===")
    x_cmp = np.sin(np.linspace(0, 4*np.pi, 200)) + 0.2*np.random.randn(200)
    print("DE (mixta):", entropy_dispersion(x_cmp, d=3, tau=1, c=6))
    print("PE (mixta):", entropy_permuta(x_cmp, m=3, tau=1))

# -------------------------------------------------
# Ejecutar testeo
# -------------------------------------------------
test_entropies()


=== 1. Serie constante ===
DE (const): -3.03440049141233e-13
PE (const): -5.581602429106793e-13

=== 2. Serie aleatoria uniforme ===
DE (random): 0.9574806450803466
PE (random): 0.9958247165318197

=== 3. Serie senoidal ===
DE (sinusoidal): 0.5031380541975659
PE (sinusoidal): 0.4967356215755387

=== 4. Sensibilidad a parámetros ===
DE (d=2): 0.8022
DE (d=3): 0.7773
DE (d=4): 0.7286
PE (m=2): 1.0000
PE (m=3): 0.9972
PE (m=4): 0.9872

=== 5. Comparación en señal mixta (sin+ruido) ===
DE (mixta): 0.6435679277903418
PE (mixta): 0.996928687422783
